In [1]:
# from flask import Flask, request, jsonify
import os
from Audio2text.separate_audio import *
from Audio2text import diarize_model
from Denoiser.denoiser_run import denoise_audio
from S2T import *
from Speech_emo.emotion_predict import *
from Speech_emo import *
import time

c:\Users\Admin\anaconda3\envs\xlstm\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Admin\anaconda3\envs\xlstm\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
c:\Users\Admin\anaconda3\envs\xlstm\lib\inspect.py:869: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This 


Device set to use cpu


In [2]:
from PhoBertCNN.phobert import CNN,encoder_generator, predict_sentence, load_phobert_cnn, predict_text_emo

phoBertCNN, cnn = load_phobert_cnn(r'C:\Users\Admin\Desktop\Desktop\PhoBertCnn\savefolder\phoberttask2a_2.pt', r'C:\Users\Admin\Desktop\Desktop\PhoBertCnn\savefolder\cnntask2a_2.pt', 'cpu')


c:\Users\Admin\Desktop\Desktop\aidemo\aidemo\PhoBertCNN\phobert.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  phoBert = torch.load(phobert_path, map_location=device)

In [3]:
file_save_path = "C:/Users/Admin/Desktop/Desktop/aidemo/emodata/emodata/b/recordings (36)_clip_4.wav"

In [4]:
start = time.time()
denoised_output_folder = "C:/Users/Admin/Desktop/Desktop/aidemo/test"

# step 1: denoise audio
print("------Denosing audio------")
audio_denoised_link = denoise_audio(audio_link=file_save_path, output_folder=denoised_output_folder)
    
    # step 2: diaries audio using speaker diarization whisperx
print("------Diarizing audio------")
audio = whisperx.load_audio(audio_denoised_link) # load and embed audio for diarization
diarize_segments = get_DiarizeSegments(diarize_model = diarize_model,audio=audio, min_speakers = 2, max_speakers=2)
print(diarize_segments)
print('-----Segmenting audio-----')
threshold = 600.0
diarize_segments['duration'] = diarize_segments['end'] - diarize_segments['start']
diarize_segments = diarize_segments[diarize_segments['duration'] >= threshold]

segment_path = "C:/Users/Admin/Desktop/Desktop/aidemo/test"
diarize_segments = cut_and_save_segments(audio_link=audio_denoised_link, diarize_segments=diarize_segments, format_type = 'wav', save_path = segment_path)

# print(diarize_segments)
# bonus: combine audio
# print("------Combining audio------")
# audio_combine = combine_audio(diarize_segments, save_path= "H://Learning Files/Project AI/Aidemo/aidemo/Audio_saving_file/Combined/")
# step 3: speech emotion
print("------Predicting emotion------")
diarize_segments = predict_diarize_emo(diarize_segments)

# step 4: speech to text
print("------Transcribing audio------")
diarize_segments = text_diarize_save(diarize_segments)

# step 4.5: text emotion classification
print("------Predicting emotion from text------")
diarize_segments = predict_text_emo(diarize_segments, phoBertCNN, cnn, "cpu", encoder_generator=encoder_generator)

end = time.time()
# step 5: return to client
print("-------Done--------, time: ", end-start)	


------Denosing audio------
Ran in 2.8753671646118164s
------Diarizing audio------


c:\Users\Admin\anaconda3\envs\xlstm\lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


                             segment label     speaker   start     end
0  [ 00:00:00.030 -->  00:00:00.419]     A  SPEAKER_00    30.0   420.0
1  [ 00:00:00.722 -->  00:00:01.819]     B  SPEAKER_00   720.0  1820.0
2  [ 00:00:02.342 -->  00:00:04.249]     C  SPEAKER_00  2340.0  4250.0
3  [ 00:00:05.076 -->  00:00:05.734]     D  SPEAKER_01  5080.0  5730.0
4  [ 00:00:05.279 -->  00:00:05.565]     E  SPEAKER_00  5280.0  5570.0
5  [ 00:00:05.734 -->  00:00:09.008]     F  SPEAKER_00  5730.0  9010.0
6  [ 00:00:05.920 -->  00:00:06.544]     G  SPEAKER_01  5920.0  6540.0
-----Segmenting audio-----
Saved segment_1.wav
Saved segment_2.wav
Saved segment_3.wav
Saved segment_4.wav
Saved segment_5.wav
------Predicting emotion------
1/1 [==============================] - 0s 19ms/step
Finished predicting text emotion in 0.57124924659729s
------Transcribing audio------


c:\Users\Admin\anaconda3\envs\xlstm\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1920
  warnings.warn(
c:\Users\Admin\anaconda3\envs\xlstm\lib\site-packages\transformers\models\whisper\generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
c

------Predicting emotion from text------
Finished predicting emotion in 0.11990070343017578s
-------Done--------, time:  51.172346115112305


c:\Users\Admin\anaconda3\envs\xlstm\lib\site-packages\transformers\tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [8]:
diarize_segments

,segment,label,speaker,start,end,duration,link,emotion,text,emotion_text
1,[ 00:00:00.722 --> 00:00:01.819],B,SPEAKER_00,720.0,1820.0,1100.0,C:/Users/Admin/Desktop/Desktop/aidemo/testsegm...,"[0.00015678243653383106, 5.688188253749615e-10...",{'text': 'bạn có biết tên đường hồng đô không.'},"[99.8142, 0.088789575, 0.097010195]"
2,[ 00:00:02.342 --> 00:00:04.249],C,SPEAKER_00,2340.0,4250.0,1910.0,C:/Users/Admin/Desktop/Desktop/aidemo/testsegm...,"[16.58357048034668, 2.165447767765727e-05, 79....",{'text': 'cháu biết nhưng mà cháu không giàu ở...,"[99.0192, 0.46434864, 0.51645166]"
3,[ 00:00:05.076 --> 00:00:05.734],D,SPEAKER_01,5080.0,5730.0,650.0,C:/Users/Admin/Desktop/Desktop/aidemo/testsegm...,"[5.7097117860394064e-06, 2.8175553185602435e-1...",{'text': 'yeah.'},"[99.814095, 0.14031184, 0.045598287]"
5,[ 00:00:05.734 --> 00:00:09.008],F,SPEAKER_00,5730.0,9010.0,3280.0,C:/Users/Admin/Desktop/Desktop/aidemo/testsegm...,"[0.031319454312324524, 4.003322101198137e-05, ...",{'text': 'tại vì sao lại bị tàn tiền vì vì tự ...,"[63.315, 7.362158, 29.322842]"
6,[ 00:00:05.920 --> 00:00:06.544],G,SPEAKER_01,5920.0,6540.0,620.0,C:/Users/Admin/Desktop/Desktop/aidemo/testsegm...,"[3.531297206878662, 1.996564833461889e-06, 93....",{'text': 'unk sao nó điện.'},"[99.59135, 0.22801371, 0.18064085]"


In [ ]:
emotion_avg = pd.DataFrame(diarize_segments["emotion"].tolist()).mean().tolist()
emotion_text_avg = pd.DataFrame(diarize_segments["emotion_text"].tolist()).mean().tolist()

# Tạo DataFrame kết quả
result = pd.DataFrame({
    "emotion_avg": [emotion_avg],
    "emotion_text_avg": [emotion_text_avg]
})

In [11]:
result

,emotion_avg,emotion_text_avg
0,"[4.029269926737197, 1.2736966474020292e-05, 94...","[92.3107681274414, 1.6567243337631226, 6.03250..."


In [12]:

new_emotion_labels = ['bad', 'happy', 'normal']  

def calculate_emotion_avg(emotion_avg):
    
    bad_value = sum(emotion_avg[i] for i in [0, 1, 3])
    happy_value = emotion_avg[2]  
    normal_value = emotion_avg[4] 
    return bad_value, happy_value, normal_value

def calculate_emotion_text_avg(emotion_text_avg):
    clean_value = emotion_text_avg[0]
    bad_text_value = sum(emotion_text_avg[i] for i in [1, 2])
    return clean_value, bad_text_value

new_emotion_values = []
new_emotion_text_values = []

for index, row in result.iterrows():

    bad, happy, normal = calculate_emotion_avg(row['emotion_avg'])
    new_emotion_values.append([bad, happy, normal])
    
    clean, bad_text = calculate_emotion_text_avg(row['emotion_text_avg'])
    new_emotion_text_values.append([clean, bad_text])

result['new_emotion_values'] = new_emotion_values
result['new_emotion_text_values'] = new_emotion_text_values



In [1]:
import numpy as np

# Nhãn cảm xúc
emotion_labels = ['sad', 'happy', 'angry', 'neutral', 'fear']
text_emotion_labels = ['clean', 'offensive', 'hate']

# Tạo danh sách các cảm xúc tiêu cực
negative_emotions = {'sad', 'angry', 'fear'}
negative_texts = {'offensive', 'hate'}

def classify_conversation(emotion_values, text_emotion_values):
    """
    Phân loại hội thoại dựa trên cảm xúc âm thanh và văn bản.
    
    emotion_values: Danh sách các vector cảm xúc của các segment
    text_emotion_values: Danh sách các vector cảm xúc văn bản của các segment
    """
    # Chuyển đổi emotion_values thành nhãn
    for segment in emotion_values:
        max_index = np.argmax(segment)
        if emotion_labels[max_index] in negative_emotions:
            return "bad"

    # Chuyển đổi text_emotion_values thành nhãn
    for segment in text_emotion_values:
        max_index = np.argmax(segment)
        if text_emotion_labels[max_index] in negative_texts:
            return "bad"

    return "good"

# Ví dụ với dữ liệu từ ảnh
emotion_values = [
    [24.24, 3.19e-08, 1.84e-08, 75.75, 3.02e-05],  # neutral
    [98.89, 3.44e-06, 3.62e-08, 1.10, 5.09e-05],   # happy
    [99.97, 9.87e-05, 7.60e-08, 0.02, 9.78e-05],   # happy
    [0.00072, 2.21e-11, 3.22e-08, 99.99, 2.94e-06], # neutral
    [0.00082, 9.31e-12, 1.26e-08, 99.99, 2.13e-06]  # neutral
]

text_emotion_values = [
    [99.81, 0.088, 0.097],  # clean
    [99.01, 0.464, 0.516],  # clean
    [99.81, 0.140, 0.045],  # clean
    [63.31, 7.36, 29.32],   # offensive
    [99.59, 0.228, 0.180]   # clean
]

# Gán nhãn hội thoại
result = classify_conversation(emotion_values, text_emotion_values)
print(f"Hội thoại được phân loại là: {result}")


Hội thoại được phân loại là: bad


In [14]:
def get_highest_emotion(emotion_avg):
    return max(range(len(emotion_avg)), key=lambda i: emotion_avg[i])

def process_emotion_text(emotion_text_avg, highest_emotion_avg):
   
    clean = emotion_text_avg[0]
    bad_text = sum(emotion_text_avg[i] for i in [1, 2])
    if bad_text > clean:
        return "bad" 
    else:
        if highest_emotion_avg == 2:  # happy
            return "happy"
        elif highest_emotion_avg == 4:  # neutral
            return "neutral"
        else:
            return "bad" 
        
final_labels = []

for index, row in result.iterrows():

    highest_emotion_avg = get_highest_emotion(row['emotion_avg'])
    
    label = process_emotion_text(row['emotion_text_avg'], highest_emotion_avg)
    
    final_labels.append(label)

result['final_label'] = final_labels

print(result[['final_label']])

  final_label
0       happy


In [ ]:
def fuse_convs(self):
        """Fuse parallel convolutions."""
        w = torch.zeros_like(self.conv.weight.data)
        i = [x // 2 for x in w.shape[2:]]
        w[:, :, i[0] : i[0] + 1, i[1] : i[1] + 1] = self.cv2.weight.data.clone()
        self.conv.weight.data += w
        self.__delattr__("cv2")
        self.forward = self.forward_fuse